In [ ]:
from os import listdir
from os.path import isfile, join


In [5]:
import netCDF4 as nc
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rcParams
import import_ipynb
import re
import glob

import Module_Diagnostics as MD


# import Shock_understanding_preset as Sup

In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-d

In [2]:
#if working on lx-moissard
Cluster = 'Occ/'
run_name = '20_08_18_new_big_one_0'
filepath = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/p3_files/'
filepath_nc = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/'

time = 230 #TIME
time = '%05d' % time    # Change the time to string format, needed by functions


try:
    date = re.search('Magw_(.+?)_t', glob.glob(filepath_nc+'Magw*_t'+time+'.nc')[0]).group(1)
except (IndexError, AttributeError): 
    sys.exit(f"""time_dump {time} does not appear to have data.
             Let us go to the next time_dump""")

print(f'date of the simulation (DD_MM_YY): {date}')
print(f'time dump (in 1/omega_ci): {time}')

In [ ]:
## Load B and X, Y, Z

# There might be some MemoryError. In that case, follow the guide:
# https://stackoverflow.com/questions/60563936/memoryerror-unable-to-allocate-3-78-gib-for-an-array-with-shape-802-842-1502
Magw = MD.import_data_3D(filepath_nc, date, time, 'Magw')

cwp = Magw['c_omegapi'][0]
gstep = Magw['gstep']
r_planet = Magw['r_planet'][0]

#Thomas Huret recommands shifting the axes so that the planet is indeed in (0,0,0)
#It has got something to do with lists starting at 1 instead of 0 in Fortran...?
X = np.array(np.around(Magw['x']))#+gstep[0]  
Y = np.array(np.around(Magw['y']))#+gstep[1]
Z = np.array(np.around(Magw['z']))#-gstep[2]

nx,  ny,  nz  = len(X), len(Y), len(Z)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
nx0, ny0, nz0 = ( int(np.where(abs(X)==min(abs(X)))[0]),
                  int(np.where(abs(Y)==min(abs(Y)))[0]), 
                  int(np.where(abs(Z)==min(abs(Z)))[0])  )
            
# Use memmap to alleviate RAM
# This stores big arrays on the disk, but in a way that still allows for most
# operations available on an np.array
print("storing Magnetic field in a memmap")
file_Bx = path.join(mkdtemp(), 'Bx.dat')            
Bx = np.memmap(file_Bx, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bx[:] = Magw['Bx']
file_By = path.join(mkdtemp(), 'By.dat')            
By = np.memmap(file_By, dtype='float32', mode='w+', shape=(nx,ny,nz))
By[:] = Magw['By']
file_Bz = path.join(mkdtemp(), 'Bz.dat')            
Bz = np.memmap(file_Bz, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bz[:] = Magw['Bz']
print("deleting Magw to alleviate RAM")
del Magw
B = [Bx, By, Bz]

In [3]:
posx = []
posy = []
posz = []

velx = []
vely = []
velz = []

i = 0

for p3_file in listdir(filepath):
    
    if not(".nc" in p3_file):
        continue
    
    i+=1
        
    print("Reading file", i, "/", len(listdir(filepath)))
    print("Reading ", p3_file, " ...")

    p3_data = nc.Dataset(filepath + p3_file,'r')
    
    posx = posx + list(np.array(p3_data.variables['particule_x']))
    posy = posy + list(np.array(p3_data.variables['particule_y']))
    posz = posz + list(np.array(p3_data.variables['particule_z']))

    velx = velx + list(-np.array(p3_data.variables['particule_vx']))
    vely = vely + list(-np.array(p3_data.variables['particule_vy']))
    velz = velz + list(np.array(p3_data.variables['particule_vz']))


FileNotFoundError: [Errno 2] No such file or directory: b'/data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/p3_files/p3_0000_18_08_20_t00230.nc'

In [1]:
# /!\ ugly hard coding

nx = 1500
ny = 720
nz = 660
dx = 1
dy = 1
dz = 1
px = (nx - 100)*dx
py = ny/2 * dy
pz = nz/2 * dz

def real_pos(pos_part, planet, sgn):
    return sgn*(np.array(pos_part) - planet*np.ones(len(pos_part)))

# /!\ Verify this, I think y might need s=-1. See read_netcdf_3D.py
rposx = real_pos(posx, px, -1)
rposy = real_pos(posy, py, -1)
rposz = real_pos(posz, pz,  1)

In [ ]:
def cubes(position_shock):
    
    '''Defines two cubes: one on the left and one on the right side of
       a point.'''
    
    distance = 7
    size_cubes = 6 # Goals and poles. This will be a 5 by 5 cube.
    
    x_shock = position_shock[0]
    y_shock = position_shock[1]
    z_shock = position_shock[2]
    
    center_cube = x_shock, y_shock, z_shock

    cube_x_max = x_shock + size_cubes/2
    cube_x_min = x_shock - size_cubes/2
    cube_y_max = y_shock + size_cubes/2
    cube_y_min = y_shock - size_cubes/2
    cube_z_max = z_shock + size_cubes/2
    cube_z_min = z_shock - size_cubes/2

    ix_min_left  = int(np.where(cube_x_min + distance <=X)[0][0])    
    ix_max_left  = int(np.where(cube_x_max + distance <=X)[0][0])
    ix_min_right = int(np.where(cube_x_min - distance <=X)[0][0])    
    ix_max_right = int(np.where(cube_x_max - distance <=X)[0][0])
    iy_min = int(np.where(cube_y_min<=Y)[0][0])
    iy_max = int(np.where(cube_y_max<=Y)[0][0])
    iz_min = int(np.where(cube_z_min<=Z)[0][0])
    iz_max = int(np.where(cube_z_max<=Z)[0][0])
    
    cubes = {'left':
              { 'coord': (cube_x_min+distance, cube_x_max+distance, cube_y_min, cube_y_max, cube_z_min, cube_z_max),
                'index': (ix_min_left , ix_max_left , iy_min, iy_max, iz_min, iz_max) },
             
             'right': 
              { 'coord': (cube_x_min-distance, cube_x_max-distance, cube_y_min, cube_y_max, cube_z_min, cube_z_max),
                'index': (ix_min_right, ix_max_right, iy_min, iy_max, iz_min, iz_max) }
             }
    
    return cubes

In [ ]:
#for '20_08_18_new_big_one_0', at t=230
#cube = (xmin, xmax, ymin, ymax, zmin, zmax)
position_shock_sw = (-65, 180, 0)
position_shock_bs = (-63, 135, 0)
position_shock_mp = (-50,  76, 0)

cubes_sw = cubes(position_shock_sw)
cubes_bs = cubes(position_shock_bs)
cubes_mp = cubes(position_shock_mp)

In [ ]:
def incube(point, cube):
    x, y, z = point
    xmin, xmax, ymin, ymax, zmin, zmax = cube
    return (     (xmin < x < xmax) 
             and (ymin < y < ymax)
             and (zmin < z < zmax) )

In [2]:
print("Constructing list of particles in the solar wind")

x_sw = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
y_sw = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
z_sw = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]

vA = 89.371 #Ugly hard code
vx_sw = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
vy_sw = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]
vz_sw = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_sw)]

NameError: name 'rposx' is not defined

In [ ]:
print("Constructing list of particles in the magnetosheath, near the bow shock")

x_bs = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
y_bs = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
z_bs = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]

vA = 89.371 #Ugly hard code
vx_bs = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
vy_bs = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]
vz_bs = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_bs)]

In [ ]:
print("Constructing list of particles in the magnetosheath, near the magnetopause")

x_mp = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
y_mp = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
z_mp = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]

vA = 89.371 #Ugly hard code
vx_mp = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
vy_mp = [vely[i]*vA for i in range(0, len(rposx))
    if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]
vz_mp = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cube_mp)]

In [ ]:
def avg(data):
    return np.mean(data)

def rms(data):
    return np.mean(np.sqrt( (np.array(data) - np.array(avg(data)))**2))

In [ ]:
rcParams["figure.figsize"] = [5, 3.75]

plt.close('all')
plt.figure()
plt.scatter(vx_sw, vy_sw, s = 0.05, color='black')
# plt.title("Velocity distribution in the solar wind", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
plt.axis('equal')
# plt.xlim([avg(vx_sw)+8*rms(vx_sw), avg(vx_sw)-8*rms(vx_sw)])
# plt.ylim([avg(vy_sw)-8*rms(vy_sw), avg(vy_sw)+8*rms(vy_sw)])
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.figure()
plt.scatter(vx_bs, vy_bs, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the bow shock", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
# plt.xlim([avg(vx_bs)+8*rms(vx_bs), avg(vx_bs)-8*rms(vx_bs)])
# plt.ylim([avg(vy_bs)-8*rms(vy_bs), avg(vy_bs)+8*rms(vy_bs)])
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.figure()
plt.scatter(vx_mp, vy_mp, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the magnetopause", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
plt.xlim([avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp)])
plt.ylim([avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp)])
plt.tight_layout()
plt.show()